In [3]:
from translate import Translator
import numpy as np
import preprocessor as p
import re
import codecs
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.snowball import SpanishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Load Original Training Data

In [4]:
with open("/Users/ziranmin/Desktop/Assignment5/train/english_train.text", 'r') as f:
    en_train_texts = [l.strip() for l in f]

In [5]:
with open('/Users/ziranmin/Desktop/Assignment5/train/english_train.labels', 'r') as f:
    en_train_labels = [int(l.strip()) for l in f]

In [6]:
with open("/Users/ziranmin/Desktop/Assignment5/train/spanish_train.text", 'r') as f:
    sp_train_texts = [l.strip() for l in f]

In [7]:
with open('/Users/ziranmin/Desktop/Assignment5/train/spanish_train.labels', 'r') as f:
    sp_train_labels = [int(l.strip()) for l in f]

# Load Original Testing Data

In [8]:
with open("/Users/ziranmin/Desktop/Assignment5/test/english_test.text", 'r') as f:
    en_test_texts = [l.strip() for l in f]

In [9]:
with open("/Users/ziranmin/Desktop/Assignment5/test/english_test.labels", 'r') as f:
    en_test_labels = [int(l.strip()) for l in f]

In [10]:
with open("/Users/ziranmin/Desktop/Assignment5/test/spanish_test.text", 'r') as f:
    sp_test_texts = [l.strip() for l in f]

In [11]:
with open("/Users/ziranmin/Desktop/Assignment5/test/spanish_test.labels", 'r') as f:
    sp_test_labels = [int(l.strip()) for l in f]

# Load Mapping Labels

In [10]:
with open("/Users/ziranmin/Desktop/Assignment5/mapping/english_mapping.txt", 'r') as f:
    en_mapping = [l.strip() for l in f]

In [11]:
with open("/Users/ziranmin/Desktop/Assignment5/mapping/spanish_mapping.txt", 'r') as f:
    sp_mapping = [l.strip() for l in f]

# Data Cleaning Setting up

In [87]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ziranmin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [109]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ziranmin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
#tweeter preprocessor
p.set_options(p.OPT.URL, p.OPT.SMILEY, p.OPT.MENTION)

In [13]:
#prepare to remove punctuation
translator = str.maketrans("", "", punctuation)

In [14]:
#prepare to get stem words
get_stem_en = EnglishStemmer()

# Find All Unusual Characters

In [12]:
en_train_texts_special_char = []
for i in range(len(en_train_texts)):
    en_train_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', en_train_texts[i])

In [13]:
sp_train_texts_special_char = []
for i in range(len(sp_train_texts)):
    sp_train_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', sp_train_texts[i])

In [14]:
en_test_texts_special_char = []
for i in range(len(en_test_texts)):
    en_test_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', en_test_texts[i])

In [15]:
sp_test_texts_special_char = []
for i in range(len(sp_test_texts)):
    sp_test_texts_special_char += re.sub("[A-Z]|[À-Ú]|[a-z]|[à-ú]|[0-9]|[' ']", '', sp_test_texts[i])

In [16]:
all_special_char = np.unique(en_train_texts_special_char + sp_train_texts_special_char + en_test_texts_special_char + sp_test_texts_special_char) 

In [17]:
all_special_char  = all_special_char.tolist() 

In [18]:
all_special_char += ['...']

In [19]:
special_char_single_string = ''
for i in range(len(all_special_char)):
    special_char_single_string += all_special_char[i]
    
special_char_single_string = '[' + special_char_single_string + ']'

In [20]:
special_char_single_string

'[!"#$%&()*+,-./:;=?@[\\]^_`{|}~\xa0¡£¥¦ª«¯³´·»¿ÜûüĀāăĄćğīİıŃńňōŠżžƈƖơƷƸəɛɢɩɪɬɱɴʀʔʕʻ˃ˈ˘ˢ̵̸̡̨̄͛͟ΑΒΓΔΕΖΘΙΚΛΜΝΞΟΠΡΣΤΦΧΩίεηικνξστυφωϯАБВДЙМНРСХШабвгдеийклмнопрстухцшщьюяєҒғӜأابتحخدرصضعفقكلمنهويِกขคงจชดตทนปพฟมยรลวสหอะัาิีูเแใไๆ็่้๊ခငတပမေ္ၚლᎠᎥᎬᎶᎽᎾᏁᏆ᠌ᴄᴇᴋᴍᴏᴗᴜᴥᴬᴮᴰᴱᴳᴴᴵᴸᴹᴺᴼᴿᵀᵁᵛᵞᶜᶠạởῳ\u200b\u200c\u200d–—―‘’‛“”†•…⁰€⃣℉→↓↠⇤⇥⇨∘∞≫ⓐⓑⓓⓔⓕⓖⓗⓘⓙⓛⓜⓝⓞⓟⓡⓢⓣⓨ└╳■▷▻▽○◌●◕◡◦★☆☇☉☞☡☥☻♔♕♚♡♩♪♫♬♾⚘⚯⛇⛾✓❥➵➶⠀。《》【】いおかがくけこしじすずっでとなにのはばぶぺまみょらりれわんゞアイウェエカクグコサシスゼソタダチッツデトドノハバビフブベペホポマヨラリルレロン・ーヾ㊣一下世並丹亡人仙休力十去品国坐場夏外大好媽子家布帰帶平後所新旅日時朝林森楽死泰浜浴海焼爸甄甜生界番米糖翔芋英茶虚行要角記語谦谷豐轉辺近鋪雪鮮麻鼎가갓고광구국귀그끼남년뉴늘단딸랑램럴론마밤방버범별북븐비빠사상세셀소수스아야어얼엄에여열오요욕용운워웬음이인일자장재제지천친카켓타탄탔토페포피하한할해행\uf3e0\uf8ff︎️！）ＡＥＬＭＮＲＳＴＵＶＷａｂｃｄｅｆｇｈｉｌｍｎｏｐｒｓｔｕｖｗｙ￣�𗀛𝑒𝒶𝒸𝒽𝒾𝓂𝓇𝓈𝓉🄲🄴🄻🄽🅃🅇🅐🅑🅓🅔🅗🅙🅛🅞🅢🅣🅤🅩🇦🇪🇱🇵🇷🇸🇹🇺🕆🕇🕪🖒🖓🛇\U000fe4e6\U000fe4eb...]'

# Clean English Tweets

In [23]:
def clean_english(texts):
    result = []
    for text in texts:
        #remove URL links, Smiley, and @user
        text = p.clean(text)
        #remove special characters
        text = re.sub(special_char_single_string, '', text)
        #make everything lower case
        text = text.lower()
        #remove stopwords
        text = ' '.join([i for i in text.split() if i not in (stopwords.words('english'))])
        #remove punctuation
        text = text.translate(translator)
        #change every word to stem word
        text = [get_stem_en.stem(i) for i  in word_tokenize(text)]
        result.append(' '.join(text))
    
    return result

In [24]:
en_train_texts_cleaned = clean_english(en_train_texts)

In [25]:
len(en_train_texts_cleaned)

90000

In [26]:
en_test_texts_cleaned = clean_english(en_test_texts)

In [27]:
len(en_test_texts_cleaned)

10000

In [287]:
with codecs.open("en_train_texts_cleaned.txt",'w',"utf-8") as out_fs:
    for each in en_train_texts_cleaned:
        out_fs.write(each + "\n")

In [290]:
with codecs.open("en_test_texts_cleaned.txt",'w',"utf-8") as out_fs:
    for each in en_test_texts_cleaned:
        out_fs.write(each + "\n")

# Clean Spanish Tweets

In [28]:
get_stem_sp = SpanishStemmer()

In [29]:
def clean_spanish(texts):
    result = []
    
    for text in texts:
        #remove URL links, Smiley, and @user
        text = p.clean(text)
        #remove special characters
        text = re.sub(special_char_single_string, '', text)
        #make everything lower case
        text = text.lower()
        #remove stopwords
        text = ' '.join([i for i in text.split() if i not in (stopwords.words('spanish'))])
        #remove punctuation
        text = text.translate(translator)
        #change every word to stem word
        text = [get_stem_sp.stem(i) for i  in word_tokenize(text)]
        result.append(' '.join(text))
    
    return result

In [30]:
sp_train_texts_cleaned = clean_spanish(sp_train_texts)

In [31]:
len(sp_train_texts_cleaned)

19000

In [32]:
sp_test_texts_cleaned = clean_spanish(sp_test_texts)

In [33]:
len(en_test_texts_cleaned)

10000

In [369]:
with codecs.open("sp_train_texts_cleaned.txt",'w',"utf-8") as out_fs:
    for each in sp_train_texts_cleaned:
        out_fs.write(each + "\n")

In [370]:
with codecs.open("sp_test_texts_cleaned.txt",'w',"utf-8") as out_fs:
    for each in sp_test_texts_cleaned:
        out_fs.write(each + "\n")

# English Tweet Prediction 
# TF-IDF Vectorization 

In [34]:
with open("/Users/ziranmin/Desktop/Assignment5/en_train_texts_cleaned.txt", 'r') as f:
    en_train_texts_cleaned_read = [l.strip() for l in f]

In [35]:
with open("/Users/ziranmin/Desktop/Assignment5/en_test_texts_cleaned.txt", 'r') as f:
    en_test_texts_cleaned_read = [l.strip() for l in f]

In [36]:
tf = TfidfVectorizer()

In [37]:
en_train_tf = tf.fit_transform(en_train_texts_cleaned_read)

In [38]:
en_test_tf = tf.transform(en_test_texts_cleaned_read)

# Logistics Regression

In [39]:
LR = LogisticRegression()

In [40]:
LR.fit(en_train_tf, en_train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
en_pred = LR.predict(en_test_tf)

In [42]:
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')

In [43]:
np.savetxt('gold_labels_file.txt', np.array(en_test_labels), fmt='%d')

In [44]:
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 19.835
-----
Micro F-Score: 32.56
Precision: 32.56
Recall: 32.56


# Random Forest

In [45]:
clf = RandomForestClassifier()

In [46]:
clf.fit(en_train_tf, en_train_labels) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
en_pred = clf.predict(en_test_tf)

In [48]:
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(en_test_labels), fmt='%d')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 16.864
-----
Micro F-Score: 26.98
Precision: 26.98
Recall: 26.98


# Try Different Parameters for Logistics Regression

In [49]:
for c in [0.001, 0.01, 0.1, 1, 2, 3, 4, 5]:
    print("C: " , c )
    LR = LogisticRegression(C=c)
    LR.fit(en_train_tf, en_train_labels)
    
    en_pred = LR.predict(en_test_tf)
    
    np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
    np.savetxt('gold_labels_file.txt', np.array(en_test_labels), fmt='%d')
    %run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt
    print()

C:  0.001
Macro F-Score (official): 1.79
-----
Micro F-Score: 21.8
Precision: 21.8
Recall: 21.8

C:  0.01
Macro F-Score (official): 1.912
-----
Micro F-Score: 21.81
Precision: 21.81
Recall: 21.81

C:  0.1
Macro F-Score (official): 12.669
-----
Micro F-Score: 28.74
Precision: 28.74
Recall: 28.74

C:  1
Macro F-Score (official): 19.835
-----
Micro F-Score: 32.56
Precision: 32.56
Recall: 32.56

C:  2
Macro F-Score (official): 20.898
-----
Micro F-Score: 32.71
Precision: 32.71
Recall: 32.71

C:  3
Macro F-Score (official): 21.348
-----
Micro F-Score: 32.16
Precision: 32.16
Recall: 32.16

C:  4
Macro F-Score (official): 21.559
-----
Micro F-Score: 31.81
Precision: 31.81
Recall: 31.81

C:  5
Macro F-Score (official): 21.507
-----
Micro F-Score: 31.49
Precision: 31.49
Recall: 31.49



# Spanish Tweet Prediction 
# TF-IDF Vectorization 

In [50]:
with open("/Users/ziranmin/Desktop/Assignment5/sp_train_texts_cleaned.txt", 'r') as f:
    sp_train_texts_cleaned_read = [l.strip() for l in f]

In [51]:
with open("/Users/ziranmin/Desktop/Assignment5/sp_test_texts_cleaned.txt", 'r') as f:
    sp_test_texts_cleaned_read = [l.strip() for l in f]

In [52]:
tf = TfidfVectorizer()

In [53]:
sp_train_tf = tf.fit_transform(sp_train_texts_cleaned_read)
sp_test_tf = tf.transform(sp_test_texts_cleaned_read)

# Logistics Regression

In [54]:
LR = LogisticRegression()
LR.fit(sp_train_tf, sp_train_labels)
    
sp_pred = LR.predict(sp_test_tf)
    
np.savetxt('predicted_labels_file.txt', sp_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(sp_test_labels), fmt='%d')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 8.319
-----
Micro F-Score: 28.5
Precision: 28.5
Recall: 28.5


In [55]:
for c in range(1,11):
    print("C: " , c )
    LR = LogisticRegression(C=c)
    LR.fit(sp_train_tf, sp_train_labels)
    
    sp_pred = LR.predict(sp_test_tf)
    
    np.savetxt('predicted_labels_file.txt', sp_pred, fmt='%d')
    np.savetxt('gold_labels_file.txt', np.array(sp_test_labels), fmt='%d')
    %run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt
    print()

C:  1
Macro F-Score (official): 8.319
-----
Micro F-Score: 28.5
Precision: 28.5
Recall: 28.5

C:  2
Macro F-Score (official): 11.44
-----
Micro F-Score: 29.5
Precision: 29.5
Recall: 29.5

C:  3
Macro F-Score (official): 12.087
-----
Micro F-Score: 29.0
Precision: 29.0
Recall: 29.0

C:  4
Macro F-Score (official): 13.119
-----
Micro F-Score: 29.1
Precision: 29.1
Recall: 29.1

C:  5
Macro F-Score (official): 13.591
-----
Micro F-Score: 28.5
Precision: 28.5
Recall: 28.5

C:  6
Macro F-Score (official): 13.497
-----
Micro F-Score: 28.1
Precision: 28.1
Recall: 28.1

C:  7
Macro F-Score (official): 13.417
-----
Micro F-Score: 27.7
Precision: 27.7
Recall: 27.7

C:  8
Macro F-Score (official): 13.119
-----
Micro F-Score: 27.2
Precision: 27.2
Recall: 27.2

C:  9
Macro F-Score (official): 13.025
-----
Micro F-Score: 26.9
Precision: 26.9
Recall: 26.9

C:  10
Macro F-Score (official): 12.982
-----
Micro F-Score: 26.8
Precision: 26.8
Recall: 26.8



# Random Forest

In [56]:
clf = RandomForestClassifier()

In [57]:
clf.fit(sp_train_tf, sp_train_labels) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [58]:
sp_pred = clf.predict(sp_test_tf)

In [59]:
np.savetxt('predicted_labels_file.txt', sp_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(sp_test_labels), fmt='%s')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 11.06
-----
Micro F-Score: 23.6
Precision: 23.6
Recall: 23.6


# English Tweet Prediction
# Word2Vec Embedding

In [62]:
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [63]:
words = list(model.wv.vocab)

/Users/ziranmin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [64]:
en_train_tokenize = []
all_read = en_train_texts_cleaned_read + en_test_texts_cleaned_read
for i in range(len(all_read)):
    en_train_tokenize += [word_tokenize(all_read[i])]

In [65]:
en_train_tokenize_model = Word2Vec(en_train_tokenize, min_count=1)

In [66]:
words = list(en_train_tokenize_model.wv.vocab)

In [67]:
len(words)

93385

In [68]:
en_train_embeded = []
for i in en_train_texts_cleaned_read:
    temp = np.zeros(100)
    n = 0 
    for j in i.split():
        if j in words:
            temp += en_train_tokenize_model[j]
            n += 1
    if n != 0:
        temp /= n
    en_train_embeded.append(temp)

/Users/ziranmin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [69]:
en_test_embeded = []
for i in en_test_texts_cleaned_read:
    temp = np.zeros(100)
    n = 0 
    for j in i.split():
        if j in words:
            temp += en_train_tokenize_model[j]
            n += 1
    if n != 0:
        temp /= n
    en_test_embeded.append(temp)

/Users/ziranmin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


# Logistics Regression

In [70]:
LR = LogisticRegression()
LR.fit(en_train_embeded, en_train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
en_pred = LR.predict(en_test_embeded)

In [72]:
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(en_test_labels), fmt='%d')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 11.132
-----
Micro F-Score: 26.77
Precision: 26.77
Recall: 26.77


# Random Forest

In [167]:
clf = RandomForestClassifier()
clf.fit(en_train_embeded, en_train_labels) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [168]:
en_pred = clf.predict(en_test_embeded)

In [169]:
np.savetxt('predicted_labels_file.txt', en_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(en_test_labels), fmt='%s')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 11.09
-----
Micro F-Score: 21.61
Precision: 21.61
Recall: 21.61


# English Tweet Prediction
# Word2Vec Embedding

In [170]:
sp_train_tokenize = []
all_read_sp = sp_train_texts_cleaned_read + sp_test_texts_cleaned_read
for i in range(len(all_read_sp)):
    sp_train_tokenize += [word_tokenize(all_read_sp[i])]

In [173]:
sp_train_tokenize_model = Word2Vec(sp_train_tokenize, min_count=1)   

In [ ]:
sp_words = list(sp_train_tokenize_model.wv.vocab)

# (ignore the following output for testing)

In [178]:
sp_train_embeded = []
for i in sp_train_texts_cleaned_read:
    temp = np.zeros(100)
    n = 0 
    for j in i.split():
        if j in sp_words:
            temp += sp_train_tokenize_model[j]
            n += 1
    if n != 0:
        temp /= n
    sp_train_embeded.append(temp)

/Users/ziranmin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


225
450
675
900
1125
1350
1575
1800
2025
2250
2475
2700
2925
3150
3375
3600
3825
4050
4275
4500
4725
4950
5175
5400
5625
5850
6075
6300
6525
6750
6975
7200
7425
7650
7875
8100
8325
8550
8775
9000
9225
9450
9675
9900
10125
10350
10575
10800
11025
11250
11475
11700
11925
12150
12375
12600
12825
13050
13275
13500
13725
13950
14175
14400
14625
14850
15075
15300
15525
15750
15975
16200
16425
16650
16875
17100
17325
17550
17775
18000
18225
18450
18675
18900


In [179]:
sp_test_embeded = []
for i in sp_test_texts_cleaned_read:
    temp = np.zeros(100)
    n = 0 
    for j in i.split():
        if j in sp_words:
            temp += sp_train_tokenize_model[j]
            n += 1
    if n != 0:
        temp /= n
    sp_test_embeded.append(temp)

/Users/ziranmin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


# Logistics Regression

In [181]:
LR = LogisticRegression()
LR.fit(sp_train_embeded, sp_train_labels)

sp_pred = LR.predict(sp_test_embeded)


np.savetxt('predicted_labels_file.txt', sp_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(sp_test_labels), fmt='%s')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 2.849
-----
Micro F-Score: 20.9
Precision: 20.9
Recall: 20.9


# Random Forest

In [187]:
clf = RandomForestClassifier()
clf.fit(sp_train_embeded, sp_train_labels) 
sp_pred = clf.predict(sp_test_embeded)
np.savetxt('predicted_labels_file.txt', sp_pred, fmt='%d')
np.savetxt('gold_labels_file.txt', np.array(sp_test_labels), fmt='%s')
%run scorer_semeval18.py gold_labels_file.txt predicted_labels_file.txt

Macro F-Score (official): 5.366
-----
Micro F-Score: 14.4
Precision: 14.4
Recall: 14.4
